## Akhilesh Pant (AU FTCA: MCA)

# AI-Powered Threat (Weapons) Detection

## Using Live Camera

In [ ]:
# ✅ Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2

# ✅ Dataset Loading and Preprocessing
data_dir = 'dataset12'
img_size = (224, 224)
batch_size = 64

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    zoom_range=0.4,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training')
val_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation')

# ✅ Build the Transfer Learning Model with EfficientNetB0
num_classes = train_generator.num_classes
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Callbacks for Better Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),
    ModelCheckpoint('best_threat_detection_model.h5', monitor='val_loss', save_best_only=True)
]

# ✅ Train the Model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=0
)

# ✅ Visualize Training Performance
def plot_performance(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_performance(history)

# ✅ Save the Final Model
model.save('final_threat_detection_model.h5')
print("Final model saved successfully!")

# ✅ Live Camera Threat Detection
def detect_live_threat():
    loaded_model = tf.keras.models.load_model('final_threat_detection_model.h5')
    class_labels = list(train_generator.class_indices.keys())
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.resize(frame, img_size)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = loaded_model.predict(img, verbose=0)[0]
        label_idx = np.argmax(prediction)
        label = class_labels[label_idx]
        confidence = prediction[label_idx] * 100

        color = (0, 0, 255) if label == "armed_threat" else ((0, 255, 0) if label == "unarmed_individual" else (255, 255, 0))
        cv2.putText(frame, f'{label}: {confidence:.2f}%', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow('Threat Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start live detection
detect_live_threat()
